In [ ]:
# Comprehensive Option Pricer
# 3 Methods: Black Scholes, Binomial, and Monte Carlo

In [39]:
# Packages

import numpy as np
from scipy.stats import norm
import pandas as pd
from random import random
from statistics import mean
import matplotlib.pyplot as plt

In [171]:
# Creating Class Structure

class Option:
    def __init__(self, Stock: float, Strike: float, Time: float, MarketPrice: float, CallorPut: str, EUorAM: str):
        self.Stock = Stock
        self.Strike = Strike
        self.Time = Time
        self.MarketPrice = MarketPrice
        self.CallorPut = CallorPut
        self.EUorAM = EUorAM

# Black Scholes Call or Put Model with Greeks and Implied Volatility
# VOL SURFACE

    def BSM(self, R: float, Sigma: float) -> float:

    # Inputs
        d1 = ((np.log(self.Stock / self.Strike) + (R + ((Sigma ** 2) / 2)) * self.Time) / 
                (Sigma * np.sqrt(self.Time)))
        d2 = d1 - (Sigma * np.sqrt(self.Time))

    # Prices
        C = norm.cdf(d1) * self.Stock - norm.cdf(d2) * self.Strike * np.exp(-R * self.Time)
        P = -norm.cdf(-d1) * self.Stock + norm.cdf(-d2) * self.Strike * np.exp(-R * self.Time)

    # Common Greeks 
        vega = self.Stock * norm.pdf(d1) * np.sqrt(self.Time)
        gamma = (np.exp((-d1 ** 2) / 2)) / (self.Stock * Sigma * np.sqrt(2 * np.pi * self.Time))
    
    # Call Greeks
        c_delta = norm.cdf(d1)
        c_theta = -(-R * self.Strike * np.exp(-R * self.Time) * norm.cdf(d2) - Sigma * self.Stock * 
                    norm.pdf(d1) / (2 * np.sqrt(self.Time)))
        c_rho = self.Strike * self.Time * np.exp(-R * self.Time) * norm.cdf(d2)

    # Put Greeks 
        p_delta = -norm.cdf(-d1)
        p_theta = -(R * self.Strike * np.exp(-R * self.Time) * norm.cdf(-d2) - Sigma * self.Stock * 
                    norm.pdf(d1) / (2 * np.sqrt(self.Time)))
        p_rho = -self.Strike * self.Time * np.exp(-R * self.Time) * norm.cdf(-d2)

    # List Name
        names = ["Price", "Delta", "Gamma", "Theta", "Vega", "Rho"]

    # Call Output
        if self.CallorPut == "Call":  
            results = [C, c_delta, gamma, c_theta, vega, c_rho]
            rounded_results = [round(num, 3) for num in results]
            named_results = dict(zip(names, rounded_results))              
            return named_results
        
    # Put Output
        elif self.CallorPut == "Put":
            results = [P, p_delta, gamma, p_theta, vega, p_rho]
            rounded_results = [round(num, 3) for num in results] 
            named_results = dict(zip(names, rounded_results))             
            return named_results

# Implied Volatility
    def Implied_Vol(self, R: float, Sigma: float) -> float:
        for i in range(0, 200):
            price = Option.BSM(self, R, Sigma)["Price"]
            vega = Option.BSM(self, R, Sigma)["Vega"]
            diff = self.MarketPrice - price
            if abs(diff) < 1.0e-5:
                return round(Sigma, 4)
            Sigma += diff/vega

# Binomial Call or Put Tree

    def BinT(self, U: float, D: float, R: float) -> float: 

    # Creating Stock Tree
        stock = np.zeros((self.Time + 1, self.Time + 1))
        stock[0][0] = self.Stock
        for j in range(self.Time):    
            for i in range(self.Time): 
                if stock[i][j] != 0: 
                    stock[i][j + 1] = stock[i][j] * U
                    stock[i + 1][j + 1] = stock[i][j] * D

    # Risk Neutral Probabilites            
        P = (1 + R - D) / (U - D)
        Q = 1 - P        
    
    # For Call, Terminal Payoffs
        if self.CallorPut == "Call":
            for i in range(self.Time + 1):
                stock[i][self.Time] = max(stock[i][self.Time] - self.Strike, 0)

    # Working Backwards
            if self.EUorAM == "EU": 
                for j in range(-1,-self.Time - 1, -1):    
                    for i in range(-1,-self.Time - 1, -1):       
                        stock[i - 1][j - 1] = (1 / (1 + R)) * (stock[i][j] * Q + stock[i - 1][j] * P)
                return print(f"European Binomial Call Price = ${stock[0][0]:.3f}")
            else:
                for j in range(-1,-self.Time - 1, -1):    
                    for i in range(-1,-self.Time - 1, -1):
                        stock[i - 1][j - 1] = max((stock[i - 1][j - 1] - self.Strike),
                                            (1 / (1 + R)) * (stock[i][j] * Q + stock[i - 1][j] * P))
                return print(f"American Binomial Call Price = ${stock[0][0]:.3f}")                 
                        
    # For Put, Terminal Payoffs
        elif self.CallorPut == "Put":
            for i in range(self.Time + 1):
                stock[i][self.Time] = max(-stock[i][self.Time] + self.Strike, 0)
    
    # Working Backwards
            if self.EUorAM == "EU":
                for j in range(-1,-self.Time - 1, -1):    
                    for i in range(-1,-self.Time - 1, -1):           
                        stock[i - 1][j - 1] = (1 / (1 + R)) * (stock[i][j] * Q + stock[i-1][j] * P)
                return print(f"European Binomial Put Price = ${stock[0][0]:.3f}")
            else:
                for j in range(-1,-self.Time - 1, -1):    
                    for i in range(-1,-self.Time - 1, -1):
                        stock[i - 1][j - 1] = max(-stock[i - 1][j - 1] + self.Strike, 
                                                (1 / (1 + R)) * (stock[i][j] * Q + stock[i - 1][j] * P))
                return print(f"American Binomial Put Price = ${stock[0][0]:.3f}")
                    
# Monte Carlo Call or Put Simulation
# DO visulations and American Options

    def Sim(self, R: float, Trials: int, Sigma: float) -> float:

    # Creating Dataframe
        sim_table = np.zeros((Trials - 1, 3))
        df = pd.DataFrame(data = sim_table, columns = ["Stock Price", "Call", "Put"])

    # Generating Stock Prices and Evaluating Call and Put Payoffs at Expiration
        for i in range(Trials - 1):
            df.loc[i, "Stock Price"] = (self.Stock * np.exp((R - .5 * Sigma ** 2) * self.Time + Sigma * 
                                        np.sqrt(self.Time) * np.random.normal(0,1)))
            df.loc[i, "Call"] = max(df.loc[i, "Stock Price"] - self.Strike, 0)
            df.loc[i, "Put"] = max(-df.loc[i, "Stock Price"] + self.Strike, 0)
        
    # Valuing Options by Discounting Average Payoff
        call = mean(df.loc[:, "Call"]) * np.exp(-R * self.Time)
        put = mean(df.loc[:, "Put"]) * np.exp(-R * self.Time)

    # Logic Operator Giving Desired Result
        if self.CallorPut == "Call":
            return print(f"Monte Carlo Call Price = ${call:.3f}")
        elif self.CallorPut == "Put": 
            return print(f"Monte Carlo Put Price = ${put:.3f}")  
               
# Test Run
Test = Option(100, 100, 10, 7, "Put", "AM") 
Test.Sim(.03, 1000, .3)
print(Test.BSM(.03, .3))
Test.BinT(1.1, .9, .03)
Test.Implied_Vol(.03, .3)



Monte Carlo Put Price = $20.698
{'Price': 20.235, 'Delta': -0.215, 'Gamma': 0.003, 'Theta': 0.134, 'Vega': 92.298, 'Rho': -416.945}
American Binomial Put Price = $5.003


0.1565